**Warning:** Remember that for interacting with EDI Big Data Stack you must be authenticated at the system using kinit command. For more information, read the documentation at [Authenticating with Kerberos](https://docs.edincubator.eu/big-data-stack/basic-concepts.html#authenticating-with-kerberos).

In [ ]:
%%bash
kinit -kt ~/work/$JUPYTERHUB_USER.service.keytab $JUPYTERHUB_USER@EDINCUBATOR.EU

In [ ]:
import os

# Spark2

EDI Big Data Stack includes Spark2 engine. In this tutorial, we explain how to interact with Spark2 through SparkMagic kernel provided by JupyterLab and how to submit a Python job to the cluster. For illustrating this, we are going to perform some operations over the sample dataset introduced at [HDFS](hdfs.ipybn).

## SparkMagic

[SparkMagic](https://github.com/jupyter-incubator/sparkmagic) is a Jupyter kernel which allows communicating interactively with a Spark remote cluster. Available kernels within SparkMagic are PySpark, PySpark3, Spark and SparkR.

**Note:** SparkMagic doesn’t allow importing libraries not included with Spark, Scala, R or Python by default. If you need to use additional dependencies to process your data, you could do it separately in another different job. See [Running Docker containers over YARN](https://docs.edincubator.eu/big-data-stack/tools/creating-docker-images.html#creating-docker-images) for learning how to launch your custom Docker images at the cluster.

## Yelp example

First, we will load the sample file yelp_business.csv:

In [ ]:
business_df = spark.read.csv('/samples/yelp/yelp_business/yelp_business.csv', header=True, quote='"', escape='"')
business_df.show()

See that with Spark2 we can easily load and preview a CSV file. Like in the [MapReduce & YARN example](map-reduce-yarn.ipynb), we can count how many businesses each state has:

In [ ]:
state_count = business_df.groupBy(business_df.state).count()
state_count.show()

Although some data cleaning is needed, with Spark2 we can operate over data easily. Now, we are going to sort the result and store into HDFS.

In [ ]:
sorted_state_count = state_count.sort("count", ascending=False)
sorted_state_count.show()
sorted_state_count.write.csv('/user/{username}/spark-csv-output'.format(username=os.environ.get('SPARK_USER', '')))

If we check contents of `/user/<username>/spark-csv-output`, we can see that a set of CSV files have been generated, one for each partition.

In [ ]:
%%bash
hdfs dfs -ls /user/$JUPYTERHUB_USER/spark-csv-output

In [ ]:
%%bash
hdfs dfs -cat /user/$JUPYTERHUB_USER/spark-csv-output/<some-part>.csv

In the same way, we can also export those results in JSON format:

In [ ]:
sorted_state_count.write.json('/user/{username}/spark-json-output'.format(username=os.environ.get('SPARK_USER', '')))

In [ ]:
%%bash
hdfs dfs -ls /user/$JUPYTERHUB_USER/spark-json-output

## spark-submit

In order to execute the same job in a distributed way, we are going to code the previous instructions into a Python file. You can find `yelp_example.py` inside the *spark2example* folder at examples directory.

```python

import argparse
from pyspark.sql import SparkSession

parser = argparse.ArgumentParser(description='Execute Spark2 Yelp example.')
parser.add_argument(
    '--app_name', type=str, help="Application name", default='YelpExample')
parser.add_argument('input_file', type=str, help="Input CSV file")
parser.add_argument('output_dir', type=str, help="Output directory")

args = parser.parse_args()

spark = SparkSession.builder.appName(args.app_name).getOrCreate()
business_df = spark.read.csv(args.input_file,
                             header=True, quote='"', escape='"')

state_count = business_df.groupBy(business_df.state).count()
sorted_state_count = state_count.sort("count", ascending=False)
sorted_state_count.write.csv(args.output_dir)
```

**Note:** Don’t forget to include `–master yarn` and `–deploy-mode cluster` parameters in order to compute the job in the cluster instead of locally.

**Warning:** YARN does not overwrite non empty directories. Ensure to delete `/user/<username>/spark-csv-output` directory before submiting the job again.

In [ ]:
%%bash
spark-submit --master yarn --deploy-mode cluster ~/work/examples/spark2example/yelp_example.py /samples/yelp/yelp_business/yelp_business.csv /user/$JUPYTERHUB_USER/spark-csv-output --app_name $JUPYTERHUB_USER_YelpExample

You can find more information about the job at https://edi-master.novalocal:8443/gateway/hdp/yarnuiv2/. Check `/user/<username>/spark-csv-output` directory for the results.